In [1]:
import pandas as pd
import sys
import os
sys.path.append(r'C:\Users\tommy\Developer\trend-finder\src')
from utilities.dataframe import merge_dataframes
from langchain_ollama.embeddings import OllamaEmbeddings
import numpy as np
import ast

In [2]:
dfs = os.listdir('../data')

In [3]:
dfs = [pd.read_csv(os.path.join('../data', f"{df}")) for df in dfs if 'embeds' in df]

In [4]:
df_all = merge_dataframes(dfs)

In [35]:
llama_3_2_embeddings = OllamaEmbeddings(model='llama3.2:3b')

In [36]:
my_query = "What are the most growing communities on Reddit related to programming languages"

In [37]:
query_vector = llama_3_2_embeddings.embed_query(my_query)

In [38]:
query_vector = np.array(query_vector).reshape(1, -1)

In [39]:
db_vectors = df_all['embs'].apply(lambda x: ast.literal_eval(x))

# Convert lists to NumPy array
db_embs = np.array(db_vectors.tolist())

In [40]:
db_embs = np.array(db_vectors.tolist())

In [41]:
db_embs.shape, query_vector.shape

((12703, 3072), (1, 3072))

In [49]:
# Compute cosine similarity
cos_sim = np.dot(query_vector, db_embs.T) / (np.linalg.norm(query_vector, axis=1) * np.linalg.norm(db_embs))

In [58]:
# Get top-k most similar vectors
top_k = 10
top_k_indices = np.argsort(-cos_sim.flatten())[:top_k]
print("Most similar vectors:", top_k_indices)

Most similar vectors: [ 2355  2130  4425  3004  4279 12686  3543  5529  2133  6242]


In [59]:
df_all.loc[top_k_indices] 

,description,subredditName,subscribers,createdDate,ranking,growthDay,growthWeek,growthMonth,growthAbsoluteDay,growthAbsoluteWeek,growthAbsoluteMonth,subreddit_link,prompt,embs
2355,The official Python community for Reddit! Stay...,Python,1267005,"25th January, 2008",780,+ 0.031%,+ 0.197%,+ 0.797%,+ 392 Subscribers,+ 2489 Subscribers,+ 10016 Subscribers,https://reddit.com/r/Python,['<|start|>Information about subreddit:\n ...,"[-0.007957079, -0.007742038, -0.03130074, -0.0..."
2130,A community dedicated to all things web develo...,webdev,2516043,"25th January, 2009",371,+ 0.091%,+ 0.664%,+ 2.923%,+ 2276 Subscribers,+ 16596 Subscribers,+ 71452 Subscribers,https://reddit.com/r/webdev,['<|start|>Information about subreddit:\n ...,"[-0.014921741, -0.0066986806, -0.04354306, -0...."
4425,TypeScript is a language for application-scale...,typescript,134464,"01th October, 2012",5333,+ 0.06%,+ 0.387%,+ 1.659%,+ 80 Subscribers,+ 519 Subscribers,+ 2195 Subscribers,https://reddit.com/r/typescript,['<|start|>Information about subreddit:\n ...,"[-0.006172163, 0.0001789158, -0.043815825, -0...."
3004,🐁 This is a community for our little pocket pu...,RATS,614702,"14th October, 2008",1628,+ 0.181%,+ 1.251%,+ 7.042%,+ 1108 Subscribers,+ 7596 Subscribers,+ 40438 Subscribers,https://reddit.com/r/RATS,['<|start|>Information about subreddit:\n ...,"[-0.010849534, -0.005938535, -0.04441628, -0.0..."
4279,“The Mandela Effect is when a large group of p...,MandelaEffect,353744,"12th December, 2013",2521,+ 0.057%,+ 0.328%,+ 1.825%,+ 202 Subscribers,+ 1155 Subscribers,+ 6339 Subscribers,https://reddit.com/r/MandelaEffect,"[""<|start|>Information about subreddit:\n ...","[-0.011775391, -0.00640865, -0.04915861, -0.02..."
12686,Laravel is a free and open-source PHP web fram...,laravel,88622,"14th June, 2012",7129,+ 0.051%,+ 0.369%,+ 1.112%,+ 45 Subscribers,+ 326 Subscribers,+ 975 Subscribers,https://reddit.com/r/laravel,['<|start|>Information about subreddit:\n ...,"[-0.0061963582, -0.008397462, -0.043881778, -0..."
3543,"¡Guadalajara, Guadalajara! 🎵 Hueles a tierra m...",Guadalajara,263723,"30th November, 2008",3176,+ 0.106%,+ 0.897%,+ 3.437%,+ 280 Subscribers,+ 2344 Subscribers,+ 8763 Subscribers,https://reddit.com/r/Guadalajara,['<|start|>Information about subreddit:\n ...,"[-0.011083975, -0.018955566, -0.046559576, -0...."
5529,LangChain is an open-source framework and deve...,LangChain,27412,"26th January, 2023",15707,+ 0.219%,+ 1.843%,+ 8.588%,+ 60 Subscribers,+ 496 Subscribers,+ 2168 Subscribers,https://reddit.com/r/LangChain,['<|start|>Information about subreddit:\n ...,"[-0.01757728, -0.009813331, -0.044323206, -0.0..."
2133,The community for Old School RuneScape discuss...,2007scape,1020297,"13th February, 2013",984,+ 0.099%,+ 0.647%,+ 2.915%,+ 1005 Subscribers,+ 6559 Subscribers,+ 28896 Subscribers,https://reddit.com/r/2007scape,['<|start|>Information about subreddit:\n ...,"[-0.012173029, -0.0065405928, -0.038317416, -0..."
6242,NaN,language,33356,"23th August, 2008",13860,+ 0.159%,+ 1.593%,+ 5.711%,+ 53 Subscribers,+ 523 Subscribers,+ 1802 Subscribers,https://reddit.com/r/language,['<|start|>Information about subreddit:\n ...,"[-0.004669516, -0.0135824885, -0.03257706, -0...."


In [62]:
import torch.nn.functional as F
import torch

In [63]:
# Compute cosine similarity
similarities_torch = F.cosine_similarity(torch.tensor(query_vector), torch.tensor(db_embs))

# Get top-k most similar vectors
top_k = 5
top_k_indices = similarities_torch.topk(top_k).indices
print("Most similar vectors:", top_k_indices)

Most similar vectors: tensor([2355, 2130, 4425, 3004, 4279])
